In [3]:
import os
import torch


if torch.cuda.is_available():
    
    torch.cuda.set_device(0)
    device = torch.device("cuda")
    print("Switched to GPU")
else:
    
    device = torch.device("cpu")
    print("No GPU available, using CPU")

print("Using device:", device)


No GPU available, using CPU
Using device: cpu


In [1]:
!pip install torch

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import os
import torch


if torch.cuda.is_available():
   
    torch.cuda.set_device(0)
    device = torch.device("cuda")
    print("Switched to GPU")
else:
   
    device = torch.device("cpu")
    print("No GPU available, using CPU")

print("Using device:", device)


No GPU available, using CPU
Using device: cpu


In [4]:
import torch

print("CUDA Available:", torch.cuda.is_available())
print("Number of CUDA devices:", torch.cuda.device_count())


CUDA Available: False
Number of CUDA devices: 0


In [5]:
import torch

print("PyTorch version:", torch.__version__)
import torch

print("CUDA version:", torch.version.cuda)


PyTorch version: 2.2.1+cu121
CUDA version: 12.1


In [2]:
import os

judgement_folder = os.path.join(os.getcwd(), "judgement")
summary_folder = os.path.join(os.getcwd(), "summary")

judgements = []
summaries = []

for filename in os.listdir(judgement_folder):
    with open(os.path.join(judgement_folder, filename), 'r', encoding='utf-8') as file:
        judgements.append(file.read())
    
    with open(os.path.join(summary_folder, filename), 'r', encoding='utf-8') as file:
        summaries.append(file.read())


In [3]:
!pip install transformers

Defaulting to user installation because normal site-packages is not writeable


In [4]:
pip install sentencepiece

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [5]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
from torch.utils.data import DataLoader, Dataset
from torch.optim import Adam

class LegalDataset(Dataset):
    def __init__(self, judgements, summaries, tokenizer, max_length=512):
        self.judgements = judgements
        self.summaries = summaries
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.judgements)

    def __getitem__(self, idx):
        input_text = self.judgements[idx]
        target_text = self.summaries[idx]

        encoding = self.tokenizer(input_text, max_length=self.max_length, return_tensors='pt', truncation=True, padding='max_length')
        labels = self.tokenizer(target_text, max_length=self.max_length, return_tensors='pt', truncation=True, padding='max_length').input_ids

        return {
            'input_ids': encoding['input_ids'].squeeze(),
            'attention_mask': encoding['attention_mask'].squeeze(),
            'labels': labels.squeeze()
        }

tokenizer = T5Tokenizer.from_pretrained('t5-small')
model = T5ForConditionalGeneration.from_pretrained('t5-small')

optimizer = Adam(model.parameters(), lr=1e-4)


dataset = LegalDataset(judgements, summaries, tokenizer)
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)


for epoch in range(3):
    for batch in dataloader:
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()


model.save_pretrained('fine_tuned_legal_summarizer')


/home/iiitd/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
print("Input Encoding:", input_encoding)


NameError: name 'input_encoding' is not defined

In [4]:
summary_ids = fine_tuned_legal_summarizer.generate(**input_encoding, max_length=150, use_cache=False)


NameError: name 'fine_tuned_legal_summarizer' is not defined

In [7]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

model_directory_path = 'fine_tuned_legal_summarizer'

original_model = T5ForConditionalGeneration.from_pretrained('t5-small')
original_tokenizer = T5Tokenizer.from_pretrained('t5-small')

fine_tuned_model = T5ForConditionalGeneration.from_pretrained('fine_tuned_legal_summarizer')
fine_tuned_model.save_pretrained(model_directory_path)
original_tokenizer.save_pretrained(model_directory_path)


text_file_path = '1.txt'


with open(text_file_path, 'r', encoding='utf-8') as file:
    input_text = file.read()

input_encoding = original_tokenizer(input_text, return_tensors='pt', max_length=512, truncation=True, padding=True)
summary_ids = fine_tuned_model.generate(**input_encoding, max_length=150, use_cache=False)

generated_summary = original_tokenizer.decode(summary_ids[0], skip_special_tokens=True)


print("Generated Summary:", generated_summary)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Generated Summary: The appellant was the tenant of a flat on the second floor of a building in Bombay. The tenancy was a monthly one, the rent being Rs. 215. The appellant wanted to go to the United Kingdom for treatment of his failing eye sight and he got into touch with the complainant through one Sayed for the purpose of making necessary ar rangements about the flat occupied by him in view of his intended departure. The prosecution case was that the appellant demanded a sum of Rs. 30,000 which was later on reduced to Rs. 29,500 as consideration for putting the complainant in vacant possession of the flat and an additional amount of Rs. 2,000 for the furniture and


In [8]:
output_file_path = 'generated_summary.txt'
with open(output_file_path, 'w', encoding='utf-8') as output_file:
    output_file.write(generated_summary)

print(f"Generated summary saved to {output_file_path}")

Generated summary saved to generated_summary.txt


In [10]:
import os
from transformers import T5ForConditionalGeneration, T5Tokenizer


model_directory_path = 'fine_tuned_legal_summarizer'

input_folder_path = 'test_judg'

output_folder_path = 'LLM_generated_summary'

os.makedirs(output_folder_path, exist_ok=True)


original_model = T5ForConditionalGeneration.from_pretrained('t5-small')
original_tokenizer = T5Tokenizer.from_pretrained('t5-small')


fine_tuned_model = T5ForConditionalGeneration.from_pretrained(model_directory_path)


input_files = [f for f in os.listdir(input_folder_path) if f.endswith('.txt')]

for input_file in input_files:
    
    with open(os.path.join(input_folder_path, input_file), 'r', encoding='utf-8') as file:
        input_text = file.read()

    
    input_encoding = original_tokenizer(input_text, return_tensors='pt', max_length=512, truncation=True, padding=True)
    summary_ids = fine_tuned_model.generate(**input_encoding, max_length=150, use_cache=False)

    generated_summary = original_tokenizer.decode(summary_ids[0], skip_special_tokens=True)

   
    output_file_path = os.path.join(output_folder_path, input_file)
    with open(output_file_path, 'w', encoding='utf-8') as output_file:
        output_file.write(generated_summary)

    print(f"Generated summary saved to {output_file_path}")


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Generated Summary for 266.txt: The petitioner company was the managing director of another company, and the petitioner was the managing director of the company. The Central Government made a reference under section 5(1) of the Act before the Investigation Commission appointed under Act XXX of 1947. The Commission made a report to this effect to the Central Government under section 5(4) of the Act requesting that the case of the petitioner along with the cases of the petitioner and other members of his family may be referred to the Commission for investigation. The Central Government referred these cases to the Commission under section 5(4) of the Act to the Commission for investigation. The petitioner contended that the Central Government had no jurisdiction to examine the case of the petitioner along
Generated summary saved to LLM_generated_summary/266.txt
Generated Summary for 4568.txt: The Central Health Service was formed more than two decades ago and was intended to replace the In